<a href="https://colab.research.google.com/github/ovbystrova/hse_compling/blob/main/hw7/hw7_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture

!wget -q --show-progress http://paraphraser.ru/download/get?file_id=1 -O paraphraser.zip
!unzip paraphraser.zip -d paraphraser

In [ ]:
%%capture
!pip install pymorphy2

In [1]:
from collections import Counter
import gensim
from lxml import html
import numpy as np
from nltk.corpus import stopwords
import pandas as pd
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from string import punctuation
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier



# import nltk
# nltk.download('stopwords')
m = Mystem()
morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

In [2]:
corpus_xml = html.fromstring(open('paraphraser/paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
df = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [3]:
df.head()

,text_1,text_2,label
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0
2,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0
3,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1
4,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0


In [4]:
df.label.value_counts()

0     2957
-1    2582
1     1688
Name: label, dtype: int64

# Часть 1

Векторизуйте тексты с помощью Word2vec модели, обученной самостоятельно, и с помощью модели, взятой с rusvectores (например вот этой - http://vectors.nlpl.eu/repository/20/180.zip). Обучите 2 модели по определению перефразирования на получившихся векторах и проверьте, что работает лучше. 
Word2Vec нужно обучить на отдельном корпусе (не на парафразах). Можно взять данные из семинара или любые другие. 
!!!! ВАЖНО: Оценивать модели нужно с помощью кросс-валидации (в семинаре не кросс-валидация)! Метрика - f1.

In [5]:
def normalize(text):
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]
    return ' '.join(words)

def get_embedding(text, model, dim):
    text = text.split()
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total)
        except (KeyError, ValueError):
            continue
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    return vector

In [6]:
df['text_1_norm'] = df['text_1'].apply(normalize)
df['text_2_norm'] = df['text_2'].apply(normalize)

## Word2Vec с нуля (обучается на Википедии)

In [ ]:
%%capture
!wget https://raw.githubusercontent.com/ovbystrova/hse_compling/main/hw3/data/wiki_data.txt -O data/wiki_data.txt

In [7]:
data = open('data/wiki_data.txt').read().splitlines()[:10000]
data_norm = [normalize(text) for text in data]
data_norm = [text for text in data_norm if text]

In [8]:
w2v = gensim.models.Word2Vec([text.split() for text in data_norm], size=50, sg=1)

In [9]:
dim = 50
X_text_1_w2v = np.zeros((len(df['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(df['text_2_norm']), dim))

for i, text in enumerate(df['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim)
    
for i, text in enumerate(df['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim)

X_text_w2v = np.concatenate([X_text_1_w2v, X_text_2_w2v], axis=1)
y = df['label'].values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


In [10]:
# F1-Macro
clf = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')
scores = cross_val_score(clf, X_text_w2v, y, cv=5, scoring='f1_macro')
print("%0.2f f1-score with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.44 f1-score with a standard deviation of 0.06


In [11]:
# F1-Micro
clf = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')
scores = cross_val_score(clf, X_text_w2v, y, cv=5, scoring='f1_micro')
print("%0.2f f1-score with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.45 f1-score with a standard deviation of 0.06


## Word2Vec предобученный

In [ ]:
%%capture
!wget -q --show-progress http://vectors.nlpl.eu/repository/20/180.zip -O model.zip
!unzip model.zip -d model

In [12]:
model = gensim.models.KeyedVectors.load_word2vec_format('model/model.bin', binary=True)

In [13]:
mapping = {'A': 'ADJ',
           'ADV': 'ADV',
           'ADVPRO': 'ADV',
           'ANUM': 'ADJ',
           'APRO': 'DET',
           'COM': 'ADJ',
           'CONJ': 'SCONJ',
           'INTJ': 'INTJ',
           'NONLEX': 'X',
           'NUM': 'NUM',
           'PART': 'PART',
           'PR': 'ADP',
           'S': 'NOUN',
           'SPRO': 'PRON',
           'UNKN': 'X',
           'V': 'VERB'}

In [16]:
def normalize_mystem(text):
    tokens = []
    norm_words = m.analyze(text)
    for norm_word in norm_words:
        if 'analysis' not in norm_word:
            continue
            
        if not len(norm_word['analysis']):
            lemma = norm_word['text']
            pos = 'UNKN'
        else:
            lemma = norm_word["analysis"][0]["lex"].lower().strip()
            pos = norm_word["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
        pos = mapping[pos]
        tokens.append(lemma+'_'+pos)
    return tokens

In [ ]:
df['text_1_norm_m'] = df['text_1'].apply(normalize_mystem)
df['text_2_norm_m'] = df['text_2'].apply(normalize_mystem)

In [ ]:
dim = 50
X_text_1_w2v = np.zeros((len(df['text_1_norm_m']), dim))
X_text_2_w2v = np.zeros((len(df['text_2_norm_m']), dim))

for i, text in enumerate(df['text_1_norm_m'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim)
    
for i, text in enumerate(df['text_2_norm_m'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim)

X_text_w2v = np.concatenate([X_text_1_w2v, X_text_2_w2v], axis=1)
y = df['label'].values

In [ ]:
# F1-Macro
clf = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')
scores = cross_val_score(clf, X_text_w2v, y, cv=5, scoring='f1_macro')
print("%0.2f f1-score with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

In [ ]:
# F1-Micro
clf = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')
scores = cross_val_score(clf, X_text_w2v, y, cv=5, scoring='f1_micro')
print("%0.2f f1-score with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

# Часть 2

2) Преобразуйте тексты в векторы в каждой паре 5 методами  - SVD, NMF, Word2Vec (свой и  русвекторовский), Fastext. У вас должно получиться 5 пар векторов для каждой строчки в датасете. Между векторами каждой пары вычислите косинусную близость (получится 5 чисел для каждой пары). 

SVD и NMF применяйте к данным напрямую, а w2w и fastext обучите на отдельном корпусе (как в первой части). 

In [ ]:
from sklearn.metrics.pairwise import cosine_distances

In [ ]:
def tokenize(text):
    words = [word.strip(punct) for word in text.lower().split()]
    return ' '.join(words)

In [ ]:
# Word2vec trained on wikipedia

w2v = gensim.models.Word2Vec([text.split() for text in data_norm], size=50, sg=1)
dim = 50
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim)

w2vec_cosines = cosine_distance(X_text_1_w2v, X_text_2_w2v)

In [ ]:
# Fast-Text trained on wikipedia

fast_text = gensim.models.FastText([text.split() for text in data_norm], size=50, 
                                   min_n=4, max_n=8) 
dim = 50
data['text_1_notnorm'] = data['text_1'].apply(tokenize)
data['text_2_notnorm'] = data['text_2'].apply(tokenize)

X_text_1_ft = np.zeros((len(data['text_1_notnorm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_notnorm']), dim))

for i, text in enumerate(data['text_1_notnorm'].values):
    X_text_1_ft[i] = get_embedding(text, fast_text, dim)
    
for i, text in enumerate(data['text_2_notnorm'].values):
    X_text_2_ft[i] = get_embedding(text, fast_text, dim)

fasttext_cosines = cosine_distance(X_text_1_w2v, X_text_2_w2v)

In [ ]:
# SVD
svd = TruncatedSVD(200)

X_text_1 = svd.fit_transform(tfidf.transform(data['text_1_norm']))
X_text_2 = svd.fit_transform(tfidf.transform(data['text_2_norm']))

svd_cosines = cosine_distance(X_text_1, X_text_2)

In [ ]:
# NMF
X_text_1_nmf = nmf.transform(tfidf.transform(data['text_1_norm']))
X_text_2_nmf = nmf.transform(tfidf.transform(data['text_2_norm']))

nmf_cosines = cosine_distance(X_text_1_nmf, X_text_2_nmf)

Постройте обучающую выборку из этих близостей . Обучите любую модель (Логрег, Рандом форест или что-то ещё) на этой выборке и оцените качество на кросс-валидации (используйте микросреднюю f1-меру).  Попробуйте улучить метрику, изменив параметры в методах векторизации.
!!УТОЧНЕНИЕ: модель нужно обучить сразу на всех 5 близостях, а не по 1 модели на каждой близости!

In [ ]:
cosines = np.concatenate((w2vec_cosines, fasttext_cosines, svd_cosines, nmf_cosines), axis=1)
cosines.shape

In [ ]:
clf = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')
scores = cross_val_score(clf, cosines, y, cv=5, scoring='f1_micro')
print("%0.2f f1-score with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

In [ ]:
# Перед тем как выполнить следующий код, я подкрутила параметры векторов
clf = RandomForestClassifier(n_estimators=100, max_depth=7, min_samples_leaf=15,
                             class_weight='balanced')
scores = cross_val_score(clf, cosines, y, cv=5, scoring='f1_micro')
print("%0.2f f1-score with a standard deviation of %0.2f" % (scores.mean(), scores.std()))